In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import numba as nb
from scipy.fft import dct


In [ ]:
%timeit dct(x)

In [147]:
def dct1d(x):
    n = x.size

    y = np.zeros((n), dtype=np.float64)
    ks = np.arange(0, n)

    y = x[0] + (-1)**ks * x[n-1]
    print("yo", y[3])
    for k in range(n):
        # y[k] = x[0] + (-1)**k * x[n-1]
        for i in range(1, n-1):
            if k == 3: print(x[i] * np.cos(np.pi * k * i / (n - 1)))
            y[k] += 2 * x[i] * np.cos(np.pi * k * i / (n - 1))
        # y[k] = x[0] + (-1)**k * x[n-1] + 2 * np.sum(np.array([x[i] * np.cos(np.pi * k * i / (n - 1)) for i in range(1, n-1)]))
        y[k] /= np.sqrt(2 * (n - 1))
    return y

In [229]:
@nb.jit(
    nb.float64[:](nb.float64[:]),
    nopython=True,
    fastmath=True,
)
def dct1d_new(x):
    n = x.size
    n_is_odd = n%2
    n_by_2 = n//2
    cos_arr = np.zeros((n_by_2), dtype=np.float64)
    pi_by_n = np.pi / (n-1)
    for i in range(n_by_2): cos_arr[i] = np.cos(i * pi_by_n)
    y = np.empty((n,), dtype=np.float64)
    y[::2] = x[0] + x[n-1]
    y[1::2] = x[0] - x[n-1]
    y[0] += 2 * np.sum(x[1:-1])
    y[-1] += 2 * (np.sum(x[2:-1:2]) - np.sum(x[1:-1:2]))
    if n_is_odd:
        y[n_by_2] += 2 * (np.sum(x[4:-1:4]) - np.sum(x[2:-1:4]))
    for i in range(1, n-1):
        if n_is_odd == 1 and i == n_by_2: continue
        for j in range(1, n-1):
            if n_is_odd == 1 and j == n_by_2 and i%2 == 1: continue
            f = 1
            cond1 = (i > n_by_2 - 1)
            cond2 = (j > n_by_2 - 1)
            if cond1 and cond2:
                ix = n - i - 1
                jx = n - j - 1
                if n_is_odd == 0:
                    if (i + j) % 2 == 0: f = -1
                else:
                    if (i + j) % 2 != 0: f = -1
            else:
                if cond1:
                    ix = n - i - 1
                    if j%2 != 0: f = -1
                else:
                    ix = i
                if cond2:
                    jx = n - j - 1
                    if i%2 != 0: f = -1
                else:
                    jx = j
            if jx > 1:
                p, ix = np.divmod(ix * jx, n-1)
                if p%2 != 0: f *= -1
            if ix > n_by_2 - 1:
                f *= -1
                ix = n - ix - 1
            # if i == 2: print(x[j] * f * cos_arr[ix], x[j], ix, cos_arr[ix])
            if ix >= n_by_2: continue
            y[i] += 2 * x[j] * f * cos_arr[ix]
            # yy[i, j-1] = x[j] * np.cos(np.pi * i * j / (n-1))

    y /= np.sqrt(2 * (n - 1))
    return y


In [230]:
x = np.random.rand(15)


In [231]:
print(np.allclose(dct1d_new(x), dct(x, type=1, norm="ortho", orthogonalize=False)))
print(dct1d_new(x))
# print(dct(x, type=1, norm="ortho", orthogonalize=False))
# print(dct1d(x))
# print(dct1d_new(x) - dct(x, type=1, norm="ortho", orthogonalize=False))


True
[ 2.87393709 -0.23354176  0.11598166  0.24582327 -0.03244129  0.05964392
 -0.27770637 -0.32363791 -0.30177845 -0.39728226 -0.12853325  0.16112869
  0.35521376 -0.25871664 -0.2465312 ]


In [232]:
%timeit dct1d_new(x)
%timeit dct(x, type=1, norm="ortho", orthogonalize=False)


2.26 µs ± 28.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
7.38 µs ± 490 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
"""
[[ 0.86916369  0.4149195   0.12194476  0.74677572]
 [ 0.7031682   0.12821718 -0.037683   -0.60415425]
 [ 0.26858635 -0.33567693 -0.09865538  0.23076639]
 [-0.26858635 -0.33567693  0.09865538  0.23076639]
 [-0.7031682   0.12821718  0.037683   -0.60415425]
 [-0.86916369  0.4149195  -0.12194476  0.74677572]]
"""

In [74]:
# xx = np.random.rand(9)
xx = x.copy()
z = np.zeros((xx.size, xx.size-2))
n = xx.size
for i in range(n):
    for j in range(1, n-1):
        z[i, j-1] = np.cos(np.pi * i * j / (n-1))

# print(np.round(z, 5))

zz = np.zeros((n, n-2))
for i in range(1, n//2):
    for j in range(1, i+1):
        zz[i, j-1] = np.cos(np.pi * i * j / (n-1))

zz[0, 0] = 1

# print(zz)

zzz = np.zeros((n, n-2))
nby2 = n//2

for i in range(1, n-1):
    for j in range(1, n-1):
        pref = 1
        if i > nby2 - 1 and j > nby2 - 1:
            ix = n - i - 1
            jx = n - j - 1
            if n % 2 == 0:
                if (i + j) % 2 == 0: pref = -1
            else:
                if (i + j) % 2 != 0: pref = -1
        else:
            if i > nby2 - 1:
                ix = n - i - 1
                if j % 2 != 0: pref = -1
            else:
                ix = i
            if j > nby2 - 1:
                jx = n - j - 1
                if i % 2 != 0: pref = -1
            else:
                jx = j
        if jx > 1:
            p, ix = np.divmod(ix * jx, n-1)
            if p%2 != 0: pref *= -1
        if ix > nby2 - 1:
            pref *= -1
            ix = n - ix - 1
        
        zzz[i, j-1] = pref * zz[ix, 0]
        # print(i, j, ix)

if n % 2 != 0:
    if n > 2:
        zzz[nby2, 1::4] = -1.
        zzz[2::4, nby2-1] = -1.
    if n > 4:
        zzz[nby2, 3::4] = 1.
        zzz[4::4, nby2-1] = 1.

print(np.round(zzz, 5))

# print(zzz[1:-1,:]-z[1:-1,:])
print(np.allclose(zzz[1:-1,:], z[1:-1,:]))


[[ 0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.92388  0.70711  0.38268  0.      -0.38268 -0.70711 -0.92388]
 [ 0.70711 -0.      -0.70711 -1.      -0.70711 -0.       0.70711]
 [ 0.38268 -0.70711 -0.92388 -0.       0.92388  0.70711 -0.38268]
 [ 0.      -1.      -0.       1.      -0.      -1.       0.     ]
 [-0.38268 -0.70711  0.92388 -0.      -0.92388  0.70711  0.38268]
 [-0.70711 -0.       0.70711 -1.       0.70711 -0.      -0.70711]
 [-0.92388  0.70711 -0.38268  0.       0.38268 -0.70711  0.92388]
 [ 0.       0.       0.       1.       0.       0.       0.     ]]
True
